Fake news spreads like wildfire so to stop that we are going to build a model which will detect weather the news is fake or real.

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
from wordcloud import WordCloud

Data importing

In [ ]:
fake = pd.read_csv('../input/fake-and-real-news-dataset/Fake.csv', delimiter = ',')
true = pd.read_csv('../input/fake-and-real-news-dataset/True.csv', delimiter = ',')

In [ ]:
true.head()

In [ ]:
fake['sentiment']= 0
true['sentiment']= 1

dataset =pd.DataFrame()
dataset = true.append(fake)

In [ ]:
dataset.head()

In [ ]:
dataset.shape

# **Fake News Word Cloud**

In [ ]:
text = ' '.join(fake['text'].tolist())

In [ ]:
cloud = WordCloud(background_color='grey', height=2000, width=4000).generate(text)
def black_color_func(word, font_size, position,orientation,random_state=None, **kwargs):
    return("hsl(0,100%, 1%)")
cloud.recolor(color_func = black_color_func)
plt.figure(figsize=(16,8))
plt.imshow(cloud)
plt.axis('off')
plt.show()

# **Real News Word Cloud**

In [ ]:
text1 = ' '.join(true['text'].tolist())

In [ ]:
Real = WordCloud(background_color='grey', height=2000, width=4000).generate(text1)
def black_color_func(word, font_size, position,orientation,random_state=None, **kwargs):
    return("hsl(0,100%, 1%)")
Real.recolor(color_func = black_color_func)
plt.figure(figsize=(16,8))
plt.imshow(Real)
plt.axis('off')
plt.show()

Now we will clean the special charactes and all unrelated values.

In [ ]:
!pip install spacy==2.2.3
!python -m spacy download en_core_web_sm
!pip install beautifulsoup4==4.9.1
!pip install textblob==0.15.3
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall

Here we will use a preprocessing library to make our work easy.

In [ ]:
import preprocess_kgptalkie as ps

In [ ]:
dataset['title'] = dataset['title'].apply(lambda x: ps.remove_special_chars(x))
dataset['title'] = dataset['title'].apply(lambda x: str(x).lower())

## Now we will convert the words to the vector

In [ ]:
import gensim

In [ ]:
y = dataset['sentiment'].values

In [ ]:
x = [g.split() for g in dataset['title'].tolist()]

In [ ]:
print (x[0])

In [ ]:
column = ['date','subject']
dataset = dataset.drop(columns=column)
input_array=np.array(dataset['title'])

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 40000):
    review = re.sub('[^a-zA-Z]', ' ', input_array[i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 4000)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[0:40000, 2].values

In [ ]:
def tt_split(X, y, test_size=0.2):

    i = int((1 - test_size) * X.shape[0]) 
    o = np.random.permutation(X.shape[0])
    
    X_train, X_test = np.split(np.take(X,o,axis=0), [i])
    y_train, y_test = np.split(np.take(y,o), [i])
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = tt_split(x,y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# **Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier1 = LogisticRegression(solver='lbfgs', max_iter=7000)
classifier1.fit(X_train, y_train)

# Predicting the Test set results
y_predL = classifier1.predict(X_test)

from sklearn.metrics import confusion_matrix
# Making the Confusion Matrix
cm1 = confusion_matrix(y_test, y_predL)


print(cm1)

Therefore the accuracy of logistics regression is 94.76%

In [ ]:
import seaborn as sns
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True)

# **Naive Bayes**

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

print(cm)

Therefore the accuracy for Naive Bays classifier is 88.97%

# **Decision Tree**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier2 = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier2.fit(X_train, y_train)

# Predicting the Test set results
y_predD = classifier2.predict(X_test)

# Making the Confusion Matrix
cm2 = confusion_matrix(y_test, y_predD)
print(cm2)

The accurcy for descision tree is 88.75%

# **Random Forest**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier3 = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier3.fit(X_train, y_train)

# Predicting the Test set results
y_predR = classifier3.predict(X_test)

# Making the Confusion Matrix
cm3 = confusion_matrix(y_test, y_predR)

print(cm3)

The accuracy is 96.16%

# **KNN Classifier**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier4 = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier4.fit(X_train, y_train)

# Predicting the Test set results
y_predK = classifier4.predict(X_test)

# Making the Confusion Matrix
cm4 = confusion_matrix(y_test, y_predK)

print(cm4)

In [ ]:
pred = classifier1.predict(X_test)
pred

The accuracy is 62.15%